In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
ingre_list = pd.read_csv('/content/drive/My Drive/project mid/RAW_recipes.csv')['ingredients']
main_data = pd.read_csv('/content/drive/My Drive/project mid/recipes.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from itertools import chain
ingre_list = ingre_list.str.replace(r'[\[\]\'"]', '', regex=True).apply(lambda x: x.split(', '))
main_list = main_data['ingredients']
new_list = list(set(chain.from_iterable(ingre_list)))
new_list = [i for i in new_list if any(i in j for j in main_list)]

In [3]:
import re
main_data['ingredients'] = main_list.str.replace(r' \(.+?\)', '', regex=True).str.replace('  ', ' ').str.strip()
es_list = ['pinches', 'pouches', 'dashes', 'peaches', 'tomatoes', 'bunches', 'mangoes']
exception = ['skinless', 'boneless', 'leaves', 'seedless', 'strawberries']
def remove_word_tail(s):
  for i in es_list:
    s = s.replace(i, i[:-2])
  lst = re.findall(r'(?<=[\d½¾¼⅔⅓⅞⅝⅛⅜] )[^\d½¾¼⅔⅓⅞⅝⅛⅜\s]+s\b', s)
  lst = [i for i in lst if i not in exception]
  for i in lst:
    s = s.replace(i, i[:-1])
  return s
main_data['ingredients'] = main_data['ingredients'].apply(remove_word_tail).apply(lambda x: re.split(r', (?=[\d½¾¼⅔⅓⅞⅝⅛⅜])', x))

In [4]:
from fractions import Fraction

dict1 = {'½': '1/2', '¾': '3/4', '¼': '1/4', '⅔': '2/3', '⅓': '1/3', '⅞': '7/8', '⅝': '5/8', '⅛': '1/8', '⅜': '3/8'}
def change_quantity(s):
  for i in range(len(s)):
    if re.search(r'\d', s[i][0]):
      s[i] = s[i].split(maxsplit = 1)
      if s[i][1][0] in dict1:
        s[i][0] = int(s[i][0]) + Fraction(dict1[s[i][1][0]])
        s[i][1] = s[i][1].replace(f'{s[i][1][0]} ', '')
      else:
        s[i][0] = Fraction(s[i][0])
    elif re.search(r'[½¾¼⅔⅓⅞⅝⅛⅜]', s[i][0]):
      s[i] = s[i].split(maxsplit = 1)
      s[i][0] = Fraction(dict1[s[i][0]])
    else:
      s[i] = [Fraction(0), s[i]]
  return s
main_data['ingredients'] = main_data['ingredients'].apply(change_quantity)

In [5]:
main_data['ingredients'] = main_data['ingredients'].apply(lambda x: {i[1]: i[0] for i in x})

In [11]:
print(main_data['ingredients'].head())

0    {'tablespoon butter': 3, 'pound Granny Smith a...
1    {'small Granny Smith apples, or as needed': 8,...
2    {'apple - peeled, cored and chopped': 4, 'cup ...
3    {'cup all-purpose apples, peeled, cored and sl...
4    {'cup thinly sliced apples': 18, 'tablespoon l...
Name: ingredients, dtype: object
